In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt

# Define the PlotDisruption class which handles loading parameters, data, and generating the plot.
class PlotDisruption:
    
    # The __init__ method initializes the class. It loads the necessary parameters and data for plotting.
    def __init__(self, params_file, strategy_file_prefix='data/value_', figure_dir='figure/'):
        # Load parameters from the text file
        self.model, self.strategy, self.steps = self.load_parameters(params_file)
        # Load the data from the CSV file based on the strategy read from the parameters
        self.data = self.load_data(strategy_file_prefix)
        # Define the directory where the figure will be saved
        self.figure_dir = figure_dir
    
    # This method reads parameters from a text file. The file is expected to contain three lines:
    # the first line with the model name, the second with the strategy, and the third with the number of steps.
    def load_parameters(self, params_file):
        with open(params_file, 'r') as file:
            model = file.readline().strip()  # Read the model name
            strategy = file.readline().strip()  # Read the strategy
            steps = int(file.readline().strip())  # Read 'steps' as an integer
        return model, strategy, steps
    
    # This method loads data from a CSV file based on the strategy prefix provided.
    # The 'idnr' column is added as an index, and 'id_prefix' extracts the first three characters of 'id'.
    def load_data(self, file_prefix):
        data = pd.read_csv(f"{file_prefix}{self.strategy}.csv", sep=';')
        data['idnr'] = range(1, len(data) + 1)  # Create a sequential 'idnr' column starting from 1
        data['id_prefix'] = data['id'].astype(str).str[:3]  # Create 'id_prefix' from the first three characters of 'id'
        return data
    
    # This method annotates the top 'steps' points based on the values from the model's column.
    def annotate_top_steps(self, ax):
        # Select the top 'steps' values from the column specified by the model
        top = self.data.nlargest(self.steps, self.model)
        
        # If there are 100 or fewer data points, annotate them with 'id_prefix'
        if len(self.data) <= 100:
            for _, row in top.iterrows():
                ax.annotate(f'{row["id_prefix"]}', (row['idnr'], row[self.model]),
                            textcoords="offset points", xytext=(12, 0), ha='center', fontsize=14)
        # Highlight the top 'steps' points by marking them with red color
        for _, row in top.iterrows():
            ax.scatter(row['idnr'], row[self.model], color='r')
    
    # This method creates a scatter plot with the data.
    def create_scatter_plot(self):
        # Set up the figure with size 10x6 inches
        plt.figure(figsize=(10, 6))
        # Create a scatter plot using 'idnr' as the x-axis and the model's values as the y-axis
        plt.scatter(self.data['idnr'], self.data[self.model], label=self.model, color='b')

        # Configure the plot axes labels and their font sizes
        plt.xlabel('node', fontsize=14)
        plt.ylabel(f'{self.model} measures', fontsize=14)
        plt.tick_params(axis='x', labelsize=12)  # Set tick parameters for x-axis
        plt.tick_params(axis='y', labelsize=12)  # Set tick parameters for y-axis

        # Call the annotation method to highlight the top 'steps' points
        self.annotate_top_steps(plt)
        # Save the figure to the specified directory
        self.save_figure()
        # Display the plot on the screen
        plt.show()
    
    # This method saves the plot as a PNG file in the figure directory.
    def save_figure(self):
        plt.savefig(f'{self.figure_dir}key_agents.png', dpi=300)

# Main function - program execution starts here
def main():
    # Define the parameters file path
    params_file = 'data/parameters.txt'
    # Create an instance of the PlotDisruption class and pass the parameters file
    plot_disruption = PlotDisruption(params_file)
    # Generate and display the scatter plot
    plot_disruption.create_scatter_plot()

# Check if the script is being run directly
if __name__ == "__main__":
    # If so, run the main function
    main()
